In [12]:
%config Completer.use_jedi = False

import h5py
import numpy as np
import json

In [2]:
file = h5py.File('/Users/lucamasserano/Desktop/uni/cmu/ada/ada_code/data/1.hdf5', 'r')
file.keys()

<KeysViewHDF5 ['E', 'hl_x', 'meta', 'y']>

## True energy

In [3]:
true_nrg = file['y']
true_nrg

<HDF5 dataset "y": shape (1200,), type "<f4">

In [4]:
true_nrg[0]

655.69965

## High level features (obsolete version, now done in C++)

In [5]:
hl_feats = file['hl_x']
hl_feats

<HDF5 dataset "hl_x": shape (1200, 7), type "<f4">

In [6]:
hl_feats[0]

array([8.8310101e+02, 9.5349579e+02, 5.2524761e+03, 1.6910354e+04,
       3.8697146e+03, 5.7824880e-01, 7.8367944e+00], dtype=float32)

## Muons hits

Dimensions: (muon, calorimeter, z-cell, x-cell, y-cell). <br>
The dimension named 'calorimeter' apparently is just a *box* containing the actual grid 

In [7]:
hits = file['E']
hits

<HDF5 dataset "E": shape (1200, 1, 50, 32, 32), type "<f4">

#### Features V[0] and V[1]

In [8]:
hits[0][0].shape

(50, 32, 32)

In [9]:
np.where(hits[0][0] <= 0.1)

(array([ 0,  0,  0, ..., 49, 49, 49]),
 array([ 0,  0,  0, ..., 31, 31, 31]),
 array([ 0,  1,  2, ..., 29, 30, 31]))

In [10]:
np.sum(hits[0][0][np.where(hits[0][0] > 0.1)])# + np.sum(hits[0][0][np.where(hits[0][0] > 0.1)])

1.6827272

In [11]:
hits[()].squeeze().sum(axis=(1,2,3), where=(hits[()].squeeze() > 0.1)).shape

(1200,)

In [12]:
hits[()].squeeze().squeeze().shape

(1200, 50, 32, 32)

In [13]:
list(range(len(hits.shape)))[-3:]

[2, 3, 4]

In [14]:
hits[0][0].sum(axis=(0,1,2))

7.836794

## Metadata

In [15]:
metadata = json.loads(file['meta'][()])
metadata

{'e_mean': 0.000518606039858202,
 'e_sum': 31863.155088887936,
 'e_square_sum': 188868.6728268921,
 'len': 1200,
 'hl_feats': ['sume2rz_0',
  'sume2rz_1',
  'sume2rz_2',
  'sume2rz_3',
  'sume2rz_4',
  'rmom',
  'esum'],
 'hl_mean': [6001.090446607977,
  7430.592034116686,
  8066.831258693412,
  7832.179394274016,
  7573.081590550582,
  0.7213336141548932,
  26.552629240739922],
 'hl_sum': [7201308.535929573,
  8916710.440940022,
  9680197.510432094,
  9398615.273128819,
  9087697.908660699,
  865.6003369858718,
  31863.155088887906],
 'hl_square_sum': [123383767717.06184,
  163792438507.3064,
  201351317940.6949,
  186933638557.56647,
  186802666204.0862,
  866.4390811842718,
  6586960.255155739]}

## Muon 16 features

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import pylab as plt
from tqdm import tqdm
import os

import sys
sys.path.append('../muon_regression/')

In [7]:
from scripts import prep_data

In [8]:
data = pd.read_csv('./data/MuEloss_N396000_em0.1.asc', 
                   sep=' ', header=None,
                   names=[f'v[{i}]' for i in range(0, 16)] + ['true_energy'],
                   dtype=np.float128)

In [9]:
data

,v[0],v[1],v[2],v[3],v[4],v[5],v[6],v[7],v[8],v[9],v[10],v[11],v[12],v[13],v[14],v[15],true_energy
0,1.68273,6.15407,0.578249,1.269930,123.0730,46.68700,53.8146,144.41300,154.2800,104.0630,1.0,5.0,0.970702,3.0,3.0,0.442111,655.700
1,0.22672,3.22143,1.506160,1.407130,93.6621,4.45421,62.9253,37.83110,144.4230,136.3770,1.0,2.0,0.226720,0.0,0.0,0.000000,136.523
2,4.75289,9.47245,1.025090,0.150712,205.4830,630.37700,187.4360,147.50700,188.6130,174.3480,1.0,6.0,1.688000,5.0,4.0,1.251760,702.982
3,18.28610,15.39850,0.589862,0.355303,177.0400,174.67700,172.9010,153.49300,156.0430,321.6490,1.0,33.0,14.621800,6.0,6.0,1.304280,1536.120
4,6.80569,9.03234,0.933928,0.546600,159.8110,6.22464,118.5770,149.95700,340.8290,95.4652,1.0,17.0,6.159160,2.0,2.0,0.502743,1062.760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395995,47.43600,0.00000,0.573455,0.573455,175.5570,132.90900,185.6510,184.41300,100.8120,225.8470,1.0,289.0,1.636290,662.0,902.0,2.233330,1002.870
395996,7.23727,0.00000,0.905068,0.905068,121.6890,62.08430,75.9965,258.74200,98.0651,137.1370,1.0,839.0,6.739440,10.0,400.0,0.017752,920.702
395997,9.98190,0.00000,0.877561,0.877561,186.7020,34.92040,194.0400,9.71442,224.1320,68.8687,1.0,40.0,0.067988,57.0,368.0,0.676978,696.027
395998,4.50558,0.00000,0.865484,0.865484,95.1507,65.58240,43.3061,134.84000,133.0310,45.1880,1.0,529.0,4.209100,12.0,268.0,0.016958,266.856


### APPARENTLY THEY LOSE FLOAT PRECISION WHEN WRITING TO CSV! 
<b> E.G.: true_nrg[0] = 655.7 != 655.69965
    
Or maybe they lose it when passing through the C++ macro
    --> UPDATE: it seems that the C++ macro is ok. The problem is in ROOT itself: when it reads directly a root file, it collapses precision. Try loading the macro in ROOT and showing the first entry. It is already 655.7 and not 655.69965

<b> On average, roughly every three observation two are equal

In [10]:
len(data.groupby('true_energy'))

257744

In [95]:
len(data)

396000

### ADD true_energy FROM HDF5 (WHICH HAS FULL PRECISION) 

#### Read all root files into hdf5 and extract all true energies

In [33]:
with open('./use_files.txt', 'r') as usefiles:
    for file in tqdm(list(sorted(usefiles.read().split('\n')[:-1]))):
        prep_data.singleroot_to_hdf5(in_file=file, out_file=f'./data/hdf5/{file[42:][:-5]}.hdf5', write=True)

100%|██████████| 330/330 [59:06<00:00, 10.75s/it] 


In [13]:
correct_energies = []
for filename in sorted(sorted(os.listdir('./data/hdf5/'))[1:], key=lambda el: int(el[:-5])):
    file_hdf5 = h5py.File(f'./data/hdf5/{filename}', 'r')
    correct_energies.append(file_hdf5['y'][:])

In [14]:
correct_energies = np.vstack(correct_energies).reshape(-1)

In [15]:
data['correct_true_energy'] = correct_energies

In [16]:
data

,v[0],v[1],v[2],v[3],v[4],v[5],v[6],v[7],v[8],v[9],v[10],v[11],v[12],v[13],v[14],v[15],true_energy,correct_true_energy
0,1.68273,6.15407,0.578249,1.269930,123.0730,46.68700,53.8146,144.41300,154.2800,104.0630,1.0,5.0,0.970702,3.0,3.0,0.442111,655.700,655.699646
1,0.22672,3.22143,1.506160,1.407130,93.6621,4.45421,62.9253,37.83110,144.4230,136.3770,1.0,2.0,0.226720,0.0,0.0,0.000000,136.523,136.523148
2,4.75289,9.47245,1.025090,0.150712,205.4830,630.37700,187.4360,147.50700,188.6130,174.3480,1.0,6.0,1.688000,5.0,4.0,1.251760,702.982,702.981567
3,18.28610,15.39850,0.589862,0.355303,177.0400,174.67700,172.9010,153.49300,156.0430,321.6490,1.0,33.0,14.621800,6.0,6.0,1.304280,1536.120,1536.123901
4,6.80569,9.03234,0.933928,0.546600,159.8110,6.22464,118.5770,149.95700,340.8290,95.4652,1.0,17.0,6.159160,2.0,2.0,0.502743,1062.760,1062.758545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395995,47.43600,0.00000,0.573455,0.573455,175.5570,132.90900,185.6510,184.41300,100.8120,225.8470,1.0,289.0,1.636290,662.0,902.0,2.233330,1002.870,1002.868286
395996,7.23727,0.00000,0.905068,0.905068,121.6890,62.08430,75.9965,258.74200,98.0651,137.1370,1.0,839.0,6.739440,10.0,400.0,0.017752,920.702,920.701599
395997,9.98190,0.00000,0.877561,0.877561,186.7020,34.92040,194.0400,9.71442,224.1320,68.8687,1.0,40.0,0.067988,57.0,368.0,0.676978,696.027,696.027405
395998,4.50558,0.00000,0.865484,0.865484,95.1507,65.58240,43.3061,134.84000,133.0310,45.1880,1.0,529.0,4.209100,12.0,268.0,0.016958,266.856,266.855713


CORRECTLY ALIGNED.

In [14]:
data.loc[np.abs(data.true_energy - data.correct_true_energy) > 0.01, :]

,v[0],v[1],v[2],v[3],v[4],v[5],v[6],v[7],v[8],v[9],v[10],v[11],v[12],v[13],v[14],v[15],true_energy,correct_true_energy


### There are still some duplicates!

In [29]:
len(data.correct_true_energy.unique())

392613

In [26]:
data[data.duplicated(subset='correct_true_energy', keep=False)].sort_values(by='correct_true_energy', ascending=True)

,v[0],v[1],v[2],v[3],v[4],v[5],v[6],v[7],v[8],v[9],v[10],v[11],v[12],v[13],v[14],v[15],true_energy,correct_true_energy
14282,0.28499,3.14892,1.850860,0.393191,52.6928,40.42330,39.5912,0.787409,103.7560,18.5160,1.0,1.0,0.151489,1.0,1.0,0.133501,129.963,129.963470
219775,3.71440,0.00000,2.002930,2.002930,85.5209,24.55350,24.0221,144.026000,52.3162,110.8110,1.0,299.0,3.479390,6.0,220.0,0.024965,129.963,129.963470
302843,5.18890,0.00000,0.731363,0.731363,117.9590,3.46006,155.4280,182.635000,21.2636,34.1255,1.0,601.0,4.922920,13.0,209.0,0.011896,138.718,138.718475
70374,6.16174,0.00000,0.455881,0.455881,100.1040,87.22480,85.8626,73.418600,43.3046,136.3620,1.0,733.0,5.898810,7.0,257.0,0.011714,138.718,138.718475
280070,4.21296,0.00000,1.279220,1.279220,81.3125,72.25420,158.1910,15.157600,20.8813,90.8563,1.0,441.0,3.987260,9.0,219.0,0.016223,139.460,139.459991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362565,26.67790,0.00000,0.471205,0.471205,172.3270,53.84450,109.4230,166.413000,187.8790,24.4496,1.0,211.0,0.972798,499.0,509.0,1.864850,1995.560,1995.559082
298251,16.94740,0.00000,0.283213,0.283213,148.3340,129.95000,170.1700,72.453400,122.1320,101.9280,1.0,124.0,0.311108,354.0,473.0,2.593070,1997.580,1997.579102
357938,64.53500,0.00000,0.370883,0.370883,190.1110,129.04200,195.1180,196.749000,260.4960,177.7240,1.0,322.0,1.602690,947.0,892.0,1.235820,1997.580,1997.579102
130401,21.87620,0.00000,0.439889,0.439889,194.5540,202.17200,229.1790,64.373600,131.7790,200.2690,1.0,116.0,0.236380,216.0,765.0,0.313064,1997.920,1997.917725


In [28]:
data.drop(columns='true_energy').to_csv('./data/MuEloss_correct.asc', index=False, sep=' ', header=False)

## Random things

In [ ]:
pwd

In [ ]:
data = pd.read_csv('./data/MuEloss_N396000_em0.1.asc', 
                   sep=" ", header=None,
                   names=[f'v[{i}]' for i in range(0, 16)] + ['true_energy'])

In [ ]:
s = set(range(data.shape[0]))

In [ ]:
len(s)

In [ ]:
some_idxs = np.random.choice(data.index, size=1000)
some_idxs.shape

In [ ]:
s.difference_update(set(some_idxs))

In [ ]:
len(s)

In [ ]:
data.to_numpy()[list(({}))]

In [ ]:
data_arr = data.to_numpy()

In [ ]:
data_arr[np.isin(data_arr[:, -1], data_arr[:, -1][:15])][:,-1]

In [ ]:
np.where(np.isin(data_arr[:,-1], 1957.78))[0]

## Save corrected datasets

In [45]:
data = pd.read_csv('./data/MuEloss_correct.csv', sep=" ")
data["correct_true_energy"] = correct_energies

In [47]:
data.sort_values(by="correct_true_energy")

,v[0],v[1],v[2],v[3],v[4],v[5],v[6],v[7],v[8],v[9],v[10],v[11],v[12],v[13],v[14],v[15],true_energy,correct_true_energy
7923,1.14041,5.56736,2.044580,1.588830,135.0920,108.3170,148.7600,147.95900,93.0572,137.3470,1.0,4.0,0.977303,1.0,1.0,0.163107,100.003,100.002663
272900,2.80289,0.00000,1.700200,1.700200,49.1464,124.3610,37.6196,35.51300,16.7874,36.5332,1.0,176.0,2.695640,6.0,79.0,0.013866,100.007,100.007057
77607,3.22408,0.00000,1.940770,1.940770,69.6647,10.5161,24.7173,9.00178,56.1591,173.3650,1.0,251.0,3.064500,5.0,126.0,0.019142,100.009,100.009315
60472,3.64237,0.00000,1.555970,1.555970,61.4670,16.9251,70.2164,42.26420,94.0139,75.0174,1.0,291.0,3.451560,11.0,150.0,0.021268,100.010,100.010376
165447,4.63372,0.00000,1.217500,1.217500,116.0060,84.1963,141.4890,145.07600,54.5947,107.1170,1.0,470.0,4.226200,12.0,307.0,0.031327,100.011,100.011444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56124,8.11009,0.00000,0.198280,0.198280,121.2920,64.8659,78.1715,144.96600,136.5790,125.4040,1.0,943.0,7.592200,9.0,429.0,0.025769,1999.980,1999.977783
362517,19.88430,0.00000,0.289243,0.289243,175.6140,170.6840,112.2080,133.48200,298.4210,184.1480,1.0,96.0,0.215492,606.0,697.0,4.471470,1999.980,1999.981201
88746,54.19210,0.00000,0.721687,0.721687,187.3650,176.9850,180.2150,205.71000,192.7730,158.5800,1.0,243.0,2.851580,856.0,964.0,5.733980,1999.980,1999.981567
224622,42.54630,0.00000,0.645061,0.645061,177.8280,185.9960,179.3210,88.60350,188.5240,37.6589,1.0,228.0,1.109730,931.0,620.0,1.172040,1999.980,1999.983643


In [48]:
data.drop(columns="true_energy").to_csv("./data/mueloss_correct_server.asc", sep=' ', index=False, header=False, float_format='%.6f')

In [71]:
data = pd.read_csv("./data/mueloss_correct_server.asc", sep=' ', header=None).sort_values(by=16).reset_index(drop=True)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1.14041,5.56736,2.044580,1.588830,135.0920,108.3170,148.7600,147.95900,93.0572,137.3470,1.0,4.0,0.977303,1.0,1.0,0.163107,100.002663
1,2.80289,0.00000,1.700200,1.700200,49.1464,124.3610,37.6196,35.51300,16.7874,36.5332,1.0,176.0,2.695640,6.0,79.0,0.013866,100.007057
2,3.22408,0.00000,1.940770,1.940770,69.6647,10.5161,24.7173,9.00178,56.1591,173.3650,1.0,251.0,3.064500,5.0,126.0,0.019142,100.009315
3,3.64237,0.00000,1.555970,1.555970,61.4670,16.9251,70.2164,42.26420,94.0139,75.0174,1.0,291.0,3.451560,11.0,150.0,0.021268,100.010376
4,4.63372,0.00000,1.217500,1.217500,116.0060,84.1963,141.4890,145.07600,54.5947,107.1170,1.0,470.0,4.226200,12.0,307.0,0.031327,100.011444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395995,8.11009,0.00000,0.198280,0.198280,121.2920,64.8659,78.1715,144.96600,136.5790,125.4040,1.0,943.0,7.592200,9.0,429.0,0.025769,1999.977783
395996,19.88430,0.00000,0.289243,0.289243,175.6140,170.6840,112.2080,133.48200,298.4210,184.1480,1.0,96.0,0.215492,606.0,697.0,4.471470,1999.981201
395997,54.19210,0.00000,0.721687,0.721687,187.3650,176.9850,180.2150,205.71000,192.7730,158.5800,1.0,243.0,2.851580,856.0,964.0,5.733980,1999.981567
395998,42.54630,0.00000,0.645061,0.645061,177.8280,185.9960,179.3210,88.60350,188.5240,37.6589,1.0,228.0,1.109730,931.0,620.0,1.172040,1999.983643


In [72]:
v0v1_issues = pd.read_csv("./data/v0v1_issues.csv").sort_values(by="correct_true_energy")
v0v1_issues

,correct_true_energy,v0_mine,v1_mine,v0_c++,v1_c++
0,100.002662,1.140409,5.567357,1.14041,5.56736
1,100.007058,0.000000,2.802887,1.14041,5.56736
2,100.009315,0.134055,3.090026,3.22408,0.00000
3,100.010376,0.250414,3.391959,3.22408,0.00000
4,100.011445,0.255659,4.378062,4.63372,0.00000
...,...,...,...,...,...
395995,1999.977759,1.974045,6.136049,16.54140,0.00000
395996,1999.981198,8.005069,11.879238,8.11009,0.00000
395997,1999.981529,30.095099,24.096981,19.88430,0.00000
395998,1999.983646,23.893570,18.652776,42.54630,0.00000


In [73]:
data.loc[:, 0] = v0v1_issues.v0_mine
data.loc[:, 1] = v0v1_issues.v1_mine

In [81]:
shuffled_idx = np.random.permutation(data.index)

array([ 86191, 229126, 321933, ..., 200546, 356787, 260903])

In [84]:
data.iloc[shuffled_idx,:].to_csv("./data/mueloss_allcorrect_server.asc", sep=' ', index=False, header=False, float_format='%.6f')